In [1]:
import sys
sys.path.append("../../")
import fibi.src.pie_chart as pie_chart, fibi.src.statistical_test as statistical_test
import pathlib as p
import numpy as np
import tqdm
import pandas as pd
import pyarrow.parquet as pq
root = p.Path("../data/algorithms_out/tsp/data_TSP_Uniformly distr. towns_quad_insertion.parquet").resolve()
root_folder = p.Path("../").resolve()
table = pq.read_table(root)
df = table.to_pandas()
df.head()

,num_iter,num_iter_glob,init_cost,final_cost,duration,SEED_GLOB,SEED_PROBLEM,SEED_ASSIGN,DATASET,FI_BI,IMPR,NUM_TOWNS,NUM_DIM,OPT,filename,KEY,PROBLEM,init_meth,ratio
0,12.0,46.0,878.90566,414.21856,0.000466,0.0,0.0,0.0,Uniformly distr. towns,BI,RAND,20,2.0,1.0,datasetInsertion,0.0,TSP,random,0.47129
1,31.0,141.0,1532.47995,464.98636,0.001902,1.0,0.0,0.0,Uniformly distr. towns,BI,RAND,30,2.0,1.0,datasetInsertion,1.0,TSP,random,0.30342
2,51.0,50.0,878.90566,350.46373,0.000521,26.0,0.0,0.0,Uniformly distr. towns,FI,RAND,20,2.0,1.0,datasetInsertion,26.0,TSP,random,0.39875
3,30.0,200.0,1967.04181,657.27162,0.005419,2.0,0.0,0.0,Uniformly distr. towns,BI,RAND,40,2.0,1.0,datasetInsertion,2.0,TSP,random,0.33414
4,111.0,110.0,1532.47995,490.62533,0.002361,27.0,0.0,0.0,Uniformly distr. towns,FI,RAND,30,2.0,1.0,datasetInsertion,27.0,TSP,random,0.32015


In [2]:
list_cases_per_init = {}
allowed_init = {"RAND", "GREEDY", "GREEDY randomized"}
for initialization, df_init in df.groupby("IMPR"):
    if initialization not in allowed_init:
        continue
    L_cases = []
    for instance, df_instance in df_init.groupby(["NUM_TOWNS"]):
        df_instance.sort_values(by="SEED_GLOB", inplace=True)
        bi = df_instance.query("FI_BI == 'BI'")["final_cost"].to_numpy()
        fi = df_instance.query("FI_BI == 'FI'")["final_cost"].to_numpy()
        init = df_instance.query("FI_BI == 'FI'")["init_cost"].to_numpy()
        diff = (bi-fi)/init
        case = pie_chart.get_case_from_diff(
            diff=diff,
            maximization=False,
            init_random=initialization == "RAND"
        )
        L_cases.append(case)

    assert len(L_cases) > 0, "More than 0 samples needed"
    # We build the input for the pie chart function (one list asssociated to the initialization random)
    list_cases_per_init[initialization] = L_cases


problem = "tsp"
dataset = "randomUniformInsertion"
# We create the piechart
figure_text = pie_chart.make_latex_piecharts_figure_for_datasets_of_problem(
    list_cases_per_init=list_cases_per_init,
    problem=problem,
    dataset=dataset,
    template_folder=root_folder / "data" / "templates" / "latex"
)
# And save it
with open(root_folder / "data" / "out_diagrams" / f"{problem}_{dataset}.tex", "w", encoding="utf-8") as fp:
    fp.write(figure_text)

: 